# Customer Support for Trading App

1. Based on the User Query, The main Agent decides which Agent to call(Using Structured Output)
Below are the list of agents
2. Agent for calling Tools on stocks(YFinance module) and using which we get the financial information of the stock
3. Agent for using knowledge Base. genereate the PDF of 2 pages on financial statment of a company named ChinnoVino Soft. 
5. Team Agents.
   generate a DDL and insert statements -- user and his trades in the last week, u only think of all the  columns we need and make sure the information is indian related, pls think twoice and decide all the columns
   a. 1st Agent will create SQL Query based on user text.

   b. 2nd Agent will use that query and fetch the data from DB (on user previous traded information)


In [144]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from pydantic import BaseModel, Field
from phi.agent import Agent, RunResponse
from phi.utils.pprint import pprint_run_response
from phi.tools.yfinance import YFinanceTools
from phi.vectordb.pgvector import PgVector, SearchType
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.chroma import ChromaDb
from phi.storage.agent.sqlite import SqlAgentStorage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
from astrapy import DataAPIClient
import astrapy
from phi.storage.agent.sqlite import SqlAgentStorage

In [145]:
class Decider(BaseModel):
    who: str = Field(
        ...,
        description="You function as the decider: analyze the user's query and return one of the following values—'yf', 'pdf', or 'sql'."
    )


In [146]:
decider_agent = Agent(
    name="Decider Agent",
    model=OpenAIChat(id="gpt-4o",api_key=''),
    instructions=["When a question is presented, evaluate its context and decide which category it fits into. If the query is about obtaining specific stock information, return 'yf' (using the yfinancetool). If the query involves content from a PDF document like something need to be fetched from quaterly/yearly statement, return 'pdf'. If it is meant to query a database, return 'sql' "],
    response_model=Decider,
    storage=SqlAgentStorage(table_name="agent_sessions", db_file="agent_storage.db"),
    add_history_to_messages=True,
    num_history_responses=3,
)

In [147]:

knowledge_base = PDFUrlKnowledgeBase(
    urls=["apple_stock_letter.pdf"],
    vector_db=ChromaDb(collection="apple_stock_letter"),
)

In [148]:
pdf_agent = Agent(
    model=OpenAIChat(id="gpt-4o",api_key=''),
    knowledge=knowledge_base,
    add_context=True,
    search_knowledge=False,
    markdown=True,
    instructions="You are a specialized agent with expert knowledge of financial statements. For any query related to financial statements, use only the information available in your knowledge base to provide a clear and accurate response. Do not perform additional calculations or include extraneous details—simply report what is stated in the financial documents."
)

In [149]:
finance_agent = Agent(
    name="Finance Agent",
    model=OpenAIChat(id="gpt-4o",api_key=''),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
    instructions=["Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

In [150]:
import psycopg2

def get_connection():
    """
    Establishes and returns a connection to the PostgreSQL database.
    Update the connection details as per your environment.
    """
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        dbname="phidata_db",
        user="postgres",
        password="postgres"
    )
    return conn

# get_connection()

In [151]:
def get_table_schema() -> str:
    """
    Retrieves the schema for the given table name from the PostgreSQL
    information_schema and returns a formatted string listing each column,
    its data type, and (if applicable) its character maximum length.
    """
    conn = get_connection()
    table_name= "user_trades"
    try:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT column_name, data_type, character_maximum_length
                FROM information_schema.columns
                WHERE table_name = %s
                ORDER BY ordinal_position;
            """, (table_name,))
            rows = cur.fetchall()
            if not rows:
                return f"No schema found for table '{table_name}'."
            
            schema_lines = []
            for column_name, data_type, char_max_length in rows:
                if char_max_length:
                    schema_lines.append(f"{column_name}: {data_type}({char_max_length})")
                else:
                    schema_lines.append(f"{column_name}: {data_type}")
            return "\n".join(schema_lines)
    finally:
        conn.close()
# get_table_schema('user_trades')

In [152]:
def execute_sql_query(query: str) -> str:
    """
    Executes the provided SQL query in PostgreSQL and returns the results
    as a string. For SELECT queries, it returns the fetched rows;
    for DDL/DML queries, it commits the transaction and returns a success message.
    """
    conn = get_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            if cur.description:
                # This is a SELECT (or similar) query returning rows.
                rows = cur.fetchall()
                return "\n".join(str(row) for row in rows)
            else:
                # For queries like INSERT, UPDATE, or DDL commands
                conn.commit()
                return "Query executed successfully."
    except Exception as e:
        return f"Error executing query: {e}"
    finally:
        conn.close()

# execute_sql_query("select * from user_trades limit 1")

In [153]:
sql_gen = Agent(
    name="SQL_Query_Generator",
    role="An SQL Query Generator",
    model=OpenAIChat(id="gpt-4o",api_key=''),
    tools=[get_table_schema],
    instructions=[f"You are an expert SQL query generator. Using the provided table schema (schema: {get_table_schema()}  and table:user_trades), craft a precise and valid SQL query that directly addresses the user's question. Rely solely on the schema details to ensure the query is accurate and complete."],
    show_tool_calls=True,
    markdown=True,
)

In [154]:
fetch_result = Agent(
    name="fetch_result",
    role="Agent for Executing SQL Queries and Translating Database Results",
    model=OpenAIChat(id="gpt-4o",api_key=''),
    tools=[execute_sql_query],
    instructions=[f"You are an agent designed to receive SQL queries as input, execute them using the provided database tool, and then convert the raw query results into clear, natural language explanations. Your responses should be both accurate and easy for non-technical users to understand. Dont mention/explain anything about SQL Query provided"],
    show_tool_calls=True,
    markdown=True,
)

In [155]:
sql_team = Agent(
    name="SQL Team",
    model=OpenAIChat(id="gpt-4o",api_key=''),
    team=[sql_gen, fetch_result],
    instructions=[
        "First, use the SQL Query Generator to create a valid SQL query based on the user's request and the provided table schema.",
        "Then, pass the generated SQL query to the SQL Result Interpreter to execute the query and translate the results into clear, natural language.",
        "Finally, return a concise and understandable explanation of the results to the user, without mentioning any underlying SQL details."
    ],
    show_tool_calls=True,
    markdown=True,
)


# final:RunResponse = sql_team.run("Find out the distinct stock names purchased by user with ID 2")
# pprint_run_response(final, markdown=True)

In [157]:
query = input("Please type the Query::")
deciderer_response: RunResponse = decider_agent.run(query)
call = deciderer_response.content.who
if call == 'yf':
    finance_response: RunResponse = finance_agent.run(query)
    pprint_run_response(finance_response,markdown=True)
elif call =='pdf':
    pdf_response: RunResponse = pdf_agent.run(query)
    pprint_run_response(pdf_response,markdown=True)
elif call =='sql':
    sql_response: RunResponse = sql_team.run(query)
    pprint_run_response(sql_response,markdown=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Running:                                                                                                        │
│                                                                                                                 │
│  • transfer_task_to_sql_query_generator(task_description=Generate an SQL query to find the stock purchased by   │
│    user 4., expected_output=A valid SQL query that returns the stock purchased by user 4.,                      │
│    additional_information=...)                                                                                  │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • transfer_task_to_fetch_result(task_description=Execute the SQL query to find the stock purchased by user 4., │
│    expected_output=The stock details purchased by user 4 in a readable format., additional_information=SQL      │
│    Query: SELECT stock_purchased FROM user_trades WHERE user_id = 4;)                                           │
│                                                                                                                 │
│ User 4 has purchased stock in HDFC.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯